### Section 6.2: Hyperparameter Tuning for Optimizing GCN Performance

Hyperparameter tuning is essential for maximizing the performance of GCN models in NLP tasks. Given the variety of hyperparameters that affect GCN performance—such as learning rate, hidden dimension size, dropout rate, and number of layers—systematically experimenting with these parameters can lead to improved accuracy, faster convergence, and better generalization.

**Contents:**

1. **Key Hyperparameters to Tune in GCNs**
2. **Setting Up a Grid Search for Hyperparameter Tuning**
3. **Using Random Search for Efficient Tuning**
4. **Automated Tuning with Bayesian Optimization**
5. **Analyzing Results and Selecting the Best Model**
6. **Code Walkthrough**

---



### 1. Key Hyperparameters to Tune in GCNs

Here are the most important hyperparameters to tune when optimizing GCN models:

- **Learning Rate**: Controls the step size for each update during optimization. A lower learning rate often leads to more stable training but requires more epochs.
- **Hidden Dimension Size**: Determines the feature size in hidden layers, affecting the model’s capacity to capture information.
- **Dropout Rate**: A regularization parameter that randomly deactivates nodes during training, helping to prevent overfitting.
- **Number of Layers**: Defines the depth of the model. More layers allow nodes to aggregate information from further neighbors but may lead to over-smoothing if too many layers are used.
- **Batch Size**: Relevant for larger datasets, where mini-batch training can speed up optimization and smooth gradients.



### 2. Setting Up a Grid Search for Hyperparameter Tuning

A **grid search** is a systematic way to search through predefined values for each hyperparameter and evaluate each possible combination.


Code taken from the section 5.2, pre for this section codes

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import torch
import spacy

# Load the spaCy model for POS tagging and embeddings
nlp = spacy.load("en_core_web_sm")

# Define vocabulary and one-hot encoding function
vocab = ["The", "cat", "sat", "on", "the", "mat"]
vocab_dict = {word: i for i, word in enumerate(vocab)}

def one_hot_encode(sentence_tokens, vocab_dict):
    features = []
    for token in sentence_tokens:
        one_hot = [0] * len(vocab_dict)
        if token in vocab_dict:
            one_hot[vocab_dict[token]] = 1
        features.append(one_hot)
    return np.array(features)

def pos_tag_features(sentence):
    doc = nlp(sentence)
    pos_tags = [token.pos_ for token in doc]
    unique_tags = list(set(pos_tags))
    pos_dict = {tag: i for i, tag in enumerate(unique_tags)}
    features = []
    for tag in pos_tags:
        one_hot = [0] * len(pos_dict)
        one_hot[pos_dict[tag]] = 1
        features.append(one_hot)
    return np.array(features)

def word_embedding_features(sentence):
    doc = nlp(sentence)
    features = [token.vector for token in doc]
    return np.array(features)

def create_combined_features(sentence, vocab_dict):
    doc = nlp(sentence)
    sentence_tokens = [token.text for token in doc]
    one_hot_feats = one_hot_encode(sentence_tokens, vocab_dict)
    pos_feats = pos_tag_features(sentence)
    embedding_feats = word_embedding_features(sentence)
    combined_feats = np.concatenate((one_hot_feats, pos_feats, embedding_feats), axis=1)
    return combined_feats


# Create an adjacency matrix with self-loops for the sentence
def create_adjacency_matrix_with_loops(sentence):
    doc = nlp(sentence)
    num_tokens = len(doc)
    adj_matrix = np.zeros((num_tokens, num_tokens), dtype=int)
    for token in doc:
        adj_matrix[token.i][token.head.i] = 1
        adj_matrix[token.head.i][token.i] = 1
    np.fill_diagonal(adj_matrix, 1)
    return adj_matrix

# Example sentence
sentence = "The cat sat on the mat."
combined_features = create_combined_features(sentence, vocab_dict)


adj_matrix_with_loops = create_adjacency_matrix_with_loops(sentence)

# Convert data to PyTorch tensors
node_features = torch.tensor(combined_features, dtype=torch.float32)
adj_matrix = torch.tensor(adj_matrix_with_loops, dtype=torch.float32)
label = torch.tensor([1], dtype=torch.long)  # Example label (1 for positive, 0 for negative)

def aggregate_nodes(node_outputs, method="mean"):
    if method == "mean":
        # Mean pooling: averages all node embeddings, providing a balanced representation
        return node_outputs.mean(dim=0, keepdim=True)
    elif method == "sum":
        # Sum pooling: sums all node embeddings, which can give more weight to longer sentences
        return node_outputs.sum(dim=0, keepdim=True)
    elif method == "max":
        # Max pooling: selects the maximum value for each feature across all nodes
        # Max returns a tuple (values, indices), so we take the values
        return node_outputs.max(dim=0, keepdim=True)[0]
    else:
        raise ValueError("Unsupported aggregation method. Choose 'mean', 'sum', or 'max'.")



# Define a custom GCN Layer
class GCNLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(GCNLayer, self).__init__()
        self.linear = nn.Linear(in_features, out_features)

    def forward(self, node_features, adj_matrix):
        # Apply linear transformation to node features
        transformed_features = self.linear(node_features)

        # Aggregate features from neighboring nodes using adjacency matrix
        aggregated_features = torch.matmul(adj_matrix, transformed_features)

        # Normalize by node degrees to handle varying numbers of neighbors
        degree_matrix = adj_matrix.sum(dim=1, keepdim=True)
        normalized_features = aggregated_features / degree_matrix

        # Apply non-linearity
        return F.relu(normalized_features)

# Define the DeepGCNModel with variable layers
class DeepGCNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=2, dropout_rate=0.0):
        """
        Initializes a GCN model with a configurable number of layers and dropout.

        Parameters:
        - input_dim (int): Dimension of input features for each node.
        - hidden_dim (int): Dimension of hidden layer.
        - output_dim (int): Dimension of the output layer (number of classes).
        - num_layers (int): Number of GCN layers in the model.
        - dropout_rate (float): Dropout rate to apply after each GCN layer.
        """
        super(DeepGCNModel, self).__init__()

        # Initialize the list of layers
        layers = [GCNLayer(input_dim, hidden_dim)]

        # Add intermediate layers based on num_layers
        for _ in range(num_layers - 1):
            layers.append(GCNLayer(hidden_dim, hidden_dim))

        # Final layer maps hidden_dim to output_dim for classification
        layers.append(GCNLayer(hidden_dim, output_dim))

        # Convert list of layers to nn.ModuleList for PyTorch compatibility
        self.layers = nn.ModuleList(layers)

        # Dropout layer to prevent overfitting
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, node_features, adj_matrix):
        """
        Forward pass through the GCN model.

        Parameters:
        - node_features (Tensor): Input feature matrix for nodes.
        - adj_matrix (Tensor): Adjacency matrix representing graph structure.

        Returns:
        - Tensor: Output embeddings for nodes.
        """
        x = node_features
        for layer in self.layers[:-1]:
            x = layer(x, adj_matrix)     # Pass through each layer
            x = self.dropout(x)          # Apply dropout after each layer

        # Final layer without dropout, as it's typically the output layer
        x = self.layers[-1](x, adj_matrix)
        return x


# Define input and output dimensions based on your data
input_dim = node_features.shape[1]  # Number of features per node
output_dim = 2  # Number of output classes, adjust this based on your task




#### Code Example: Setting Up a Grid Search



In [13]:
from sklearn.model_selection import ParameterGrid
import torch.optim as optim

criterion = nn.CrossEntropyLoss()    # Loss function


# Define the hyperparameter grid for tuning
param_grid = {
    "learning_rate": [0.001, 0.01, 0.1],  # Different learning rates to test
    "hidden_dim": [8, 16, 32],            # Different hidden dimensions for the model
    "dropout_rate": [0.0, 0.3, 0.5],      # Different dropout rates to prevent overfitting
    "num_layers": [2, 3]                  # Number of GCN layers in the model
}

# Initialize a dictionary to store results for each hyperparameter combination
grid_results = {}

# Generate all possible combinations of hyperparameters using ParameterGrid
for params in ParameterGrid(param_grid):
    print(f"\nTesting configuration: {params}")

    # Step 1: Define the model based on the current set of parameters
    # We use the specified hidden dimension and number of layers for the GCN model
    model = DeepGCNModel(input_dim, params['hidden_dim'], output_dim, num_layers=params['num_layers'])

    # Step 2: Set up the optimizer with the specified learning rate
    optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])

    # Step 3: Apply the dropout rate to the model (if implemented in the model definition)
    model.dropout_rate = params['dropout_rate']

    epochs = 10  # Number of training epochs
    # Step 4: Train the model (simplified training loop for demonstration)
    # The training loop iterates over a fixed number of epochs to minimize loss
    for epoch in range(epochs):
        # Forward pass: Get node embeddings from the GCN model
        node_outputs = model(node_features, adj_matrix)

        # Aggregate node embeddings to get a sentence-level representation
        sentence_output = aggregate_nodes(node_outputs, method="mean")

        # Compute the loss using the criterion (e.g., CrossEntropyLoss)
        loss = criterion(sentence_output, label)

        # Backward pass: Perform backpropagation and optimization step
        optimizer.zero_grad()  # Clear previous gradients
        loss.backward()        # Compute gradients
        optimizer.step()       # Update model parameters

    # Step 5: Evaluate the model after training
    # Perform a forward pass to get the final outputs after training
    with torch.no_grad():
        node_outputs = model(node_features, adj_matrix)
        sentence_output = aggregate_nodes(node_outputs, method="mean")

        # Get the predicted label (class with the highest score)
        _, predicted = torch.max(sentence_output, dim=1)

        # Store the loss and prediction in the results dictionary
        # The hyperparameter combination is used as the key
        grid_results[tuple(params.items())] = {"Loss": loss.item(), "Prediction": predicted.item()}

        # Print the loss and predicted label for this configuration
        print(f"Loss: {loss.item():.4f}, Prediction: {predicted.item()}")



Testing configuration: {'dropout_rate': 0.0, 'hidden_dim': 8, 'learning_rate': 0.001, 'num_layers': 2}
Loss: 0.6853, Prediction: 1

Testing configuration: {'dropout_rate': 0.0, 'hidden_dim': 8, 'learning_rate': 0.001, 'num_layers': 3}
Loss: 0.6940, Prediction: 0

Testing configuration: {'dropout_rate': 0.0, 'hidden_dim': 8, 'learning_rate': 0.01, 'num_layers': 2}
Loss: 0.1041, Prediction: 1

Testing configuration: {'dropout_rate': 0.0, 'hidden_dim': 8, 'learning_rate': 0.01, 'num_layers': 3}
Loss: 0.5571, Prediction: 1

Testing configuration: {'dropout_rate': 0.0, 'hidden_dim': 8, 'learning_rate': 0.1, 'num_layers': 2}
Loss: 0.0000, Prediction: 1

Testing configuration: {'dropout_rate': 0.0, 'hidden_dim': 8, 'learning_rate': 0.1, 'num_layers': 3}
Loss: 0.0000, Prediction: 1

Testing configuration: {'dropout_rate': 0.0, 'hidden_dim': 16, 'learning_rate': 0.001, 'num_layers': 2}
Loss: 0.6420, Prediction: 1

Testing configuration: {'dropout_rate': 0.0, 'hidden_dim': 16, 'learning_rate': 


### 3. Using Random Search for Efficient Tuning

Random search samples random combinations of hyperparameters from the search space, providing a faster alternative to grid search, especially when the parameter space is large.



#### Code Example: Random Search


In [14]:
import random

# Define ranges for each hyperparameter
param_ranges = {
    "learning_rate": [0.001, 0.01, 0.1],  # Possible learning rates to explore
    "hidden_dim": [8, 16, 32],            # Possible dimensions for hidden layers
    "dropout_rate": [0.0, 0.3, 0.5],      # Different dropout rates to reduce overfitting
    "num_layers": [2, 3]                  # Number of GCN layers to experiment with
}

# Specify the number of random configurations to sample
n_samples = 5

# Dictionary to store results for each sampled configuration
random_results = {}

# Loop to generate and test random configurations
for _ in range(n_samples):
    # Randomly select a value for each hyperparameter
    params = {key: random.choice(values) for key, values in param_ranges.items()}
    print(f"\nTesting configuration: {params}")

    # Step 1: Define the model with the randomly sampled parameters
    model = DeepGCNModel(input_dim, params['hidden_dim'], output_dim, num_layers=params['num_layers'])

    # Step 2: Set up the optimizer using the selected learning rate
    optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])

    # Step 3: Apply the dropout rate to the model if it has a dropout layer
    model.dropout_rate = params['dropout_rate']

    # Step 4: Train the model (simplified training loop)
    for epoch in range(epochs):
        # Forward pass: Calculate node embeddings using the GCN model
        node_outputs = model(node_features, adj_matrix)

        # Aggregate node embeddings to create a sentence-level representation
        sentence_output = aggregate_nodes(node_outputs, method="mean")

        # Compute the loss for the sentence representation
        loss = criterion(sentence_output, label)

        # Backward pass: Perform backpropagation and update the model parameters
        optimizer.zero_grad()  # Clear previous gradients
        loss.backward()        # Compute gradients
        optimizer.step()       # Update model parameters

    # Step 5: Evaluate the model after training
    with torch.no_grad():
        # Forward pass to get the final node embeddings after training
        node_outputs = model(node_features, adj_matrix)
        sentence_output = aggregate_nodes(node_outputs, method="mean")

        # Get the predicted label by finding the class with the highest score
        _, predicted = torch.max(sentence_output, dim=1)

        # Store the loss and prediction for this configuration in random_results
        random_results[tuple(params.items())] = {"Loss": loss.item(), "Prediction": predicted.item()}

        # Print the loss and predicted label for this configuration
        print(f"Loss: {loss.item():.4f}, Prediction: {predicted.item()}")



Testing configuration: {'learning_rate': 0.1, 'hidden_dim': 8, 'dropout_rate': 0.0, 'num_layers': 2}
Loss: 0.6931, Prediction: 0

Testing configuration: {'learning_rate': 0.001, 'hidden_dim': 16, 'dropout_rate': 0.3, 'num_layers': 2}
Loss: 0.6931, Prediction: 0

Testing configuration: {'learning_rate': 0.01, 'hidden_dim': 8, 'dropout_rate': 0.5, 'num_layers': 2}
Loss: 0.0748, Prediction: 1

Testing configuration: {'learning_rate': 0.1, 'hidden_dim': 16, 'dropout_rate': 0.0, 'num_layers': 2}
Loss: 0.6931, Prediction: 0

Testing configuration: {'learning_rate': 0.01, 'hidden_dim': 32, 'dropout_rate': 0.5, 'num_layers': 3}
Loss: 0.6931, Prediction: 0




### 4. Automated Tuning with Bayesian Optimization

Bayesian Optimization is a more sophisticated method that models the relationship between hyperparameters and model performance to intelligently explore the search space. Libraries such as **Hyperopt** or **Optuna** are commonly used for this approach.



#### Code Example: Hyperparameter Tuning with Optuna (if library is installed)


In [16]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.9 MB/s eta 0:00:00


In [17]:
import optuna

# Define the objective function that Optuna will use to optimize the hyperparameters
def objective(trial):
    # Step 1: Suggest values for each hyperparameter from specified ranges
    # Log-uniform sampling for learning rate: values between 1e-4 and 1e-1
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)

    # Categorical choice for hidden dimensions: selects either 8, 16, or 32
    hidden_dim = trial.suggest_categorical('hidden_dim', [8, 16, 32])

    # Categorical choice for dropout rate: selects either 0.0, 0.3, or 0.5
    dropout_rate = trial.suggest_categorical('dropout_rate', [0.0, 0.3, 0.5])

    # Integer choice for the number of GCN layers: selects either 2 or 3
    num_layers = trial.suggest_int('num_layers', 2, 3)

    # Step 2: Define the model with the suggested hyperparameters
    model = DeepGCNModel(input_dim, hidden_dim, output_dim, num_layers=num_layers)

    # Define the optimizer with the suggested learning rate
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Apply the suggested dropout rate to the model
    model.dropout_rate = dropout_rate

    # Step 3: Training loop (simplified version for demonstration)
    for epoch in range(epochs):
        # Forward pass: Generate node embeddings
        node_outputs = model(node_features, adj_matrix)

        # Aggregate node embeddings to get a sentence-level representation
        sentence_output = aggregate_nodes(node_outputs, method="mean")

        # Compute the loss for the sentence representation
        loss = criterion(sentence_output, label)

        # Backward pass: Calculate gradients and update model parameters
        optimizer.zero_grad()  # Clear gradients
        loss.backward()        # Calculate gradients
        optimizer.step()       # Update parameters

    # Step 4: Return the final loss for this configuration as the metric to minimize
    return loss.item()

# Run the optimization process with Optuna
# "direction='minimize'" tells Optuna to minimize the objective function (i.e., minimize the loss)
study = optuna.create_study(direction="minimize")

# Optimize the objective function across a specified number of trials
study.optimize(objective, n_trials=10)

# Print out the best hyperparameters and the corresponding loss found during the optimization
print("Best hyperparameters:", study.best_params)
print("Best loss:", study.best_value)


[I 2024-11-06 01:04:57,257] A new study created in memory with name: no-name-1d174447-4c80-4f77-b08e-5da2e3ef7742
<ipython-input-17-2842c8c02b84>:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-1)
[I 2024-11-06 01:04:57,288] Trial 0 finished with value: 3.814689989667386e-06 and parameters: {'learning_rate': 0.05843969270250119, 'hidden_dim': 8, 'dropout_rate': 0.0, 'num_layers': 2}. Best is trial 0 with value: 3.814689989667386e-06.
[I 2024-11-06 01:04:57,360] Trial 1 finished with value: 0.7060726881027222 and parameters: {'learning_rate': 0.0009953093439217103, 'hidden_dim': 8, 'dropout_rate': 0.3, 'num_layers': 2}. Best is trial 0 with value: 3.814689989667386e-06.
[I 2024-11-06 01:04:57,440] Trial 2 finished with value: 0.00036221143091097474 and parameter

Best hyperparameters: {'learning_rate': 0.09648664145833907, 'hidden_dim': 16, 'dropout_rate': 0.0, 'num_layers': 2}
Best loss: 0.0



### 5. Analyzing Results and Selecting the Best Model

After completing hyperparameter tuning, analyze the results to select the best-performing model configuration.



#### Steps:
1. **Compare Loss and Accuracy**: Identify configurations with the lowest loss and highest accuracy.
2. **Consider Training Stability**: Ensure that the selected configuration yields stable and consistent performance across epochs.
3. **Evaluate Overfitting**: Check if models with high complexity (e.g., high hidden dimensions, many layers) show signs of overfitting, especially if loss is much lower on training than validation.



#### Code Example: Summarizing Results


In [18]:
import pandas as pd

# Step 1: Convert the grid search results into a pandas DataFrame
# This will allow us to compare configurations in a tabular format easily
grid_results_df = pd.DataFrame([
    {
        "Configuration": config,       # Stores the configuration as a dictionary
        "Loss": result["Loss"],        # Stores the corresponding loss value
        "Prediction": result["Prediction"]  # Stores the predicted label for each configuration
    }
    for config, result in grid_results.items()  # Loop through each configuration-result pair
])

# Step 2: Display the best configuration based on minimum loss
# Use DataFrame's idxmin() to find the row index with the lowest loss value
best_config = grid_results_df.loc[grid_results_df["Loss"].idxmin()]

# Print the best configuration
print("Best Configuration:\n", best_config)


Best Configuration:
 Configuration    ((dropout_rate, 0.0), (hidden_dim, 8), (learni...
Loss                                                           0.0
Prediction                                                       1
Name: 4, dtype: object



### 6. Code Walkthrough: Complete Hyperparameter Tuning Pipeline

Here’s the complete code for setting up a hyperparameter tuning pipeline using grid search, random search, or Optuna (as preferred).



In [19]:
import torch.optim as optim

# Define function to run training and evaluation on the best configuration
def run_best_configuration(best_params):
    """
    Trains and evaluates the model using the best hyperparameters found.

    Parameters:
    - best_params (dict): Dictionary of best hyperparameters for the model.
    """

    print("\nTraining with Best Configuration:")
    print(best_params)  # Display the best configuration for clarity

    # Step 1: Initialize the model with the best parameters
    model = DeepGCNModel(
        input_dim,                             # Input dimension of the node features
        best_params['hidden_dim'],             # Hidden layer dimension from best configuration
        output_dim,                            # Output dimension for classification
        num_layers=best_params['num_layers']   # Number of layers as per the best configuration
    )

    # Step 2: Define the optimizer with the best learning rate
    optimizer = optim.Adam(model.parameters(), lr=best_params['learning_rate'])

    # Step 3: Set the dropout rate for the model if applicable
    model.dropout_rate = best_params['dropout_rate']

    # Training loop
    for epoch in range(epochs):
        # Step 4: Forward pass - get node outputs from the model
        node_outputs = model(node_features, adj_matrix)

        # Step 5: Aggregate node outputs to get a sentence-level embedding
        sentence_output = aggregate_nodes(node_outputs, method="mean")

        # Step 6: Compute loss
        loss = criterion(sentence_output, label)

        # Step 7: Backward pass and optimization
        optimizer.zero_grad()     # Clear previous gradients
        loss.backward()           # Backpropagate the loss
        optimizer.step()          # Update model parameters

    # Final evaluation on the best configuration
    with torch.no_grad():
        # Perform the forward pass again to evaluate model performance
        node_outputs = model(node_features, adj_matrix)
        sentence_output = aggregate_nodes(node_outputs, method="mean")

        # Get the predicted label based on the highest output score
        _, predicted = torch.max(sentence_output, dim=1)

        # Print final loss and prediction for the best configuration
        print(f"Final Loss: {loss.item():.4f}, Predicted Label: {predicted.item()}")

# Example usage after tuning:
# - For Optuna, use `study.best_params`.
# - For grid or random search, use `best_config`.
run_best_configuration(study.best_params if 'study' in globals() else best_config)



Training with Best Configuration:
{'learning_rate': 0.09648664145833907, 'hidden_dim': 16, 'dropout_rate': 0.0, 'num_layers': 2}
Final Loss: 0.6931, Predicted Label: 0



### Summary and Key Takeaways

- **Hyperparameter Tuning**: Essential for optimizing GCNs, as it significantly impacts performance, stability, and convergence speed.
- **Grid vs. Random vs. Bayesian Optimization**: Different strategies offer trade-offs in terms of efficiency, coverage, and complexity.
- **Automated Tools**: Libraries like Optuna streamline the tuning process, allowing for intelligent exploration of large parameter spaces.
- **Evaluating and Analyzing Results**: Summarizing results in a tabular format makes it easier to select the optimal configuration and identify trends among hyperparameters.

Hyperparameter tuning provides a structured approach to optimizing GCN models, enhancing their ability to capture relationships in text and improving performance across NLP tasks. With an optimized model, you can apply it confidently to various applications, knowing it’s tuned to achieve the best possible results.